# BERT finetuning tasks in 5 minutes with Cloud TPU

<table class="tfo-notebook-buttons" align="left" >
 <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>


**BERT**, or **B**idirectional **E**mbedding **R**epresentations from **T**ransformers, is a new method of pre-training language representations which obtains state-of-the-art results on a wide array of Natural Language Processing (NLP) tasks. The academic paper can be found here: https://arxiv.org/abs/1810.04805.

This Colab demonstates using a free Colab Cloud TPU to fine-tune sentence and sentence-pair classification tasks built on top of pretrained BERT models.

**Note:**  You will need a GCP (Google Compute Engine) account and a GCS (Google Cloud 
Storage) bucket for this Colab to run.

Please follow the [Google Cloud TPU quickstart](https://cloud.google.com/tpu/docs/quickstart) for how to create GCP account and GCS bucket. You have [$300 free credit](https://cloud.google.com/free/) to get started with any GCP product. You can learn more about Cloud TPU at https://cloud.google.com/tpu/docs.

Once you finish the setup, let's start!

**Firstly**, we need to set up Colab TPU running environment, verify a TPU device is succesfully connected and upload credentials to TPU for GCS bucket usage.

In [1]:
#!pip install tensorflow==1.12
!pip install tensorflow

In [2]:
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf
import urllib.request

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

from google.colab import auth
auth.authenticate_user()
with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

TPU address is grpc://10.87.50.226:8470

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 1659267899489329579),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 15448180548122047442),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 14173058163512429938),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 5563005525280413837),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 6375190459534244933),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 2645751208676096239),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 171

**Secondly**, prepare and import BERT modules. Also export data processing jupyter to python file. Use your custom bert repo! 

In [3]:
import sys

!test -d bert_repo || git clone https://github.com/chrisseiler96/bert.git bert_repo
if not 'bert_repo' in sys.path:
  sys.path += ['bert_repo']
  
#!jupyter nbconvert --to script bert_repo/data/create_train_test_datasets.ipynb

Cloning into 'bert_repo'...
remote: Enumerating objects: 329, done.
remote: Total 329 (delta 0), reused 0 (delta 0), pack-reused 329
Receiving objects: 100% (329/329), 26.69 MiB | 23.10 MiB/s, done.
Resolving deltas: 100% (183/183), done.


In [0]:
sys.path += ['bert']

**Thirdly**, prepare for training:

*  Specify task and download training data.
*  Specify BERT pretrained model
*  Specify GS bucket, create output directory for model checkpoints and eval results.



In [5]:
TASK = 'multilabel'
# Download ag_news data.

TASK_DATA_DIR = 'bert_repo/dataset'
print('***** Task data directory: {} *****'.format(TASK_DATA_DIR))

# Decompress file
#!bzip2 -d bert_repo/data/newsspace200.xml.bz
# Process data to generate train test datasets.
#!cd bert_repo/data && python create_train_test_datasets.py

# Available pretrained model checkpoints:
#   uncased_L-12_H-768_A-12: uncased BERT base model
#   uncased_L-24_H-1024_A-16: uncased BERT large model
#   cased_L-12_H-768_A-12: cased BERT large model
BERT_MODEL = 'uncased_L-12_H-768_A-12' #@param {type:"string"}
BERT_PRETRAINED_DIR = 'gs://cloud-tpu-checkpoints/bert/' + BERT_MODEL
print('***** BERT pretrained directory: {} *****'.format(BERT_PRETRAINED_DIR))
!gsutil ls $BERT_PRETRAINED_DIR

BUCKET = 'not-another-bert-bucket' #@param {type:"string"}
assert BUCKET, 'Must specify an existing GCS bucket name'
OUTPUT_DIR = 'gs://{}/bert/models/{}'.format(BUCKET, TASK)
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))


***** Task data directory: bert_repo/dataset *****
***** BERT pretrained directory: gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12 *****
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_config.json
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_model.ckpt.index
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_model.ckpt.meta
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/checkpoint
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/vocab.txt
***** Model output directory: gs://not-another-bert-bucket/bert/models/multilabel *****


**Now, let's play!**

In [7]:
# Setup task specific model and TPU running config

import modeling
import optimization
import run_multilabels_classifier
import tokenization


# Model Hyper Parameters
NUM_TRAIN_EPOCHS = 3.0
MAX_SEQ_LENGTH = 128

TRAIN_BATCH_SIZE = 32
EVAL_BATCH_SIZE = 8
LEARNING_RATE = 2e-5
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 1000
ITERATIONS_PER_LOOP = 1000
NUM_TPU_CORES = 8
VOCAB_FILE = os.path.join(BERT_PRETRAINED_DIR, 'vocab.txt')
CONFIG_FILE = os.path.join(BERT_PRETRAINED_DIR, 'bert_config.json')
INIT_CHECKPOINT = os.path.join(BERT_PRETRAINED_DIR, 'bert_model.ckpt')
DO_LOWER_CASE = BERT_MODEL.startswith('uncased')

processors = {
  #"cola": run_classifier.ColaProcessor,
  #"mnli": run_classifier.MnliProcessor,
  #"mrpc": run_classifier.MrpcProcessor,
  "multilabel": run_multilabels_classifier.MultiLabelTextProcessor,
}
processor = processors[TASK.lower()]()
#THIS IS HARDCODED
label_list = ['toxic',
              'severe_toxic',
              'obscene',
              'threat',
              'insult',
              'identity_hate']
tokenizer = tokenization.FullTokenizer(vocab_file=VOCAB_FILE, do_lower_case=DO_LOWER_CASE)

tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)
run_config = tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=OUTPUT_DIR,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=ITERATIONS_PER_LOOP,
        num_shards=NUM_TPU_CORES,
        per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))

train_examples = processor.get_train_examples(TASK_DATA_DIR)
num_train_steps = int(
    len(train_examples) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

model_fn = run_multilabels_classifier.model_fn_builder(
    bert_config=modeling.BertConfig.from_json_file(CONFIG_FILE),
    num_labels=len(label_list),
    init_checkpoint=INIT_CHECKPOINT,
    learning_rate=LEARNING_RATE,
    num_train_steps=num_train_steps,
    num_warmup_steps=num_warmup_steps,
    use_tpu=True,
    #may need to set false
    use_one_hot_embeddings=True,
    #do_serve=False
)

estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu=True,
    model_fn=model_fn,
    config=run_config,
    train_batch_size=TRAIN_BATCH_SIZE,
    eval_batch_size=EVAL_BATCH_SIZE)

INFO:tensorflow:Using config: {'_model_dir': 'gs://not-another-bert-bucket/bert/models/multilabel', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.87.50.226:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fec96d89b38>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.87.50.226:8470', '_evaluation_master': 'grpc://10.87.50.226:8470', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=1000, num_shard

In [8]:

train_file = os.path.join(OUTPUT_DIR, "train.tf_record")



run_multilabels_classifier.file_based_convert_examples_to_features(
            train_examples, label_list, MAX_SEQ_LENGTH, tokenizer, train_file)




# Train the model.
print('AG NEWS on BERT base model normally takes about 10-15 minutes. Please wait...')


#train_features = run_multilabels_classifier.convert_examples_to_features(
#    train_examples, label_list, MAX_SEQ_LENGTH, tokenizer)


print('***** Started training at {} *****'.format(datetime.datetime.now()))
print('  Num examples = {}'.format(len(train_examples)))
print('  Batch size = {}'.format(TRAIN_BATCH_SIZE))
tf.logging.info("  Num steps = %d", num_train_steps)




train_input_fn = run_multilabels_classifier.file_based_input_fn_builder(
    input_file=train_file,
    #features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=True)
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print('***** Finished training at {} *****'.format(datetime.datetime.now()))

INFO:tensorflow:Writing example 0 of 127655
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: 000103f0d9cfb60f
INFO:tensorflow:tokens: [CLS] d ' aw ##w ! he matches this background colour i ' m seemingly stuck with . thanks . ( talk ) 21 : 51 , january 11 , 2016 ( utc ) [SEP]
INFO:tensorflow:input_ids: 101 1040 1005 22091 2860 999 2002 3503 2023 4281 6120 1045 1005 1049 9428 5881 2007 1012 4283 1012 1006 2831 1007 2538 1024 4868 1010 2254 2340 1010 2355 1006 11396 1007 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0

# Eval

In [27]:
# Eval the model.
eval_examples = processor.get_dev_examples(TASK_DATA_DIR)
num_actual_eval_examples = len(eval_examples)
#while len(eval_examples) % EVAL_BATCH_SIZE != 0:
  #eval_examples.append(run_multilabels_classifier.PaddingInputExample())

eval_file = os.path.join(OUTPUT_DIR, "eval.tf_record")





eval_features = run_multilabels_classifier.file_based_convert_examples_to_features(
    eval_examples, label_list, MAX_SEQ_LENGTH, tokenizer, eval_file)




print('***** Started evaluation at {} *****'.format(datetime.datetime.now()))
print('  Num examples = {}'.format(len(eval_examples)))
print('  Batch size = {}'.format(EVAL_BATCH_SIZE))
# Eval will be slightly WRONG on the TPU because it will truncate
# the last batch.
eval_steps = int(len(eval_examples) / EVAL_BATCH_SIZE)
eval_input_fn = run_multilabels_classifier.file_based_input_fn_builder(
    input_file=eval_file,
    #features=eval_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=True)
result = estimator.evaluate(input_fn=eval_input_fn, steps=eval_steps)
print('***** Finished evaluation at {} *****'.format(datetime.datetime.now()))
output_eval_file = os.path.join(OUTPUT_DIR, "eval_results.txt")
with tf.gfile.GFile(output_eval_file, "w") as writer:
  print("***** Eval results *****")
  for key in sorted(result.keys()):
    print('  {} = {}'.format(key, str(result[key])))
    writer.write("%s = %s\n" % (key, str(result[key])))

INFO:tensorflow:Writing example 0 of 31913
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: aac73bf42ef22ff9
INFO:tensorflow:tokens: [CLS] " w ##p articles are not genealogical entries or trees , says w ##p : not . furthermore , this article is about jo ##gai ##la , and that one is about w ##ła ##dy ##sław ii of poland . our readers may find this lack of conform ##ity disturbing . - т ##р ##е ##п - " [SEP]
INFO:tensorflow:input_ids: 101 1000 1059 2361 4790 2024 2025 29606 10445 2030 3628 1010 2758 1059 2361 1024 2025 1012 7297 1010 2023 3720 2003 2055 8183 23805 2721 1010 1998 2008 2028 2003 2055 1059 22972 5149 23305 2462 1997 3735 1012 2256 8141 2089 2424 2023 3768 1997 23758 3012 14888 1012 1011 1197 16856 15290 29746 1011 1000 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 

In [24]:
# Export the model
def serving_input_fn():
  with tf.variable_scope("foo"):
    feature_spec = {
        "input_ids": tf.FixedLenFeature([MAX_SEQ_LENGTH], tf.int64),
        "input_mask": tf.FixedLenFeature([MAX_SEQ_LENGTH], tf.int64),
        "segment_ids": tf.FixedLenFeature([MAX_SEQ_LENGTH], tf.int64),
        "label_ids": tf.FixedLenFeature([6], tf.int64),
      }
    serialized_tf_example = tf.placeholder(dtype=tf.string,
                                           shape=[None],
                                           name='input_example_tensor')
    receiver_tensors = {'examples': serialized_tf_example}
    features = tf.parse_example(serialized_tf_example, feature_spec)
    return tf.estimator.export.ServingInputReceiver(features, receiver_tensors)

EXPORT_DIR = 'gs://{}/bert/export/{}'.format(BUCKET, TASK)
estimator._export_to_tpu = False  # this is important
path = estimator.export_savedmodel(EXPORT_DIR, serving_input_fn)
print(path)


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running infer on CPU
INFO:tensorflow:*** Features ***
INFO:tensorflow:  name = input_ids, shape = (?, 128)
INFO:tensorflow:  name = input_mask, shape = (?, 128)
INFO:tensorflow:  name = label_ids, shape = (?, 6)
INFO:tensorflow:  name = segment_ids, shape = (?, 128)
INFO:tensorflow:num_labels:6;logits:Tensor("loss/BiasAdd:0", shape=(?, 6), dtype=float32);labels:Tensor("loss/Cast:0", shape=(?, 6), dtype=float32)
INFO:tensorflow:**** Trainable Variables ****
INFO:tensorflow:  name = bert/embeddings/word_embeddings:0, shape = (30522, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/token_type_embeddings:0, shape = (2, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/position_embeddings:0, shape = (512, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INF

In [25]:
# 
!saved_model_cli show --all --dir gs://not-another-bert-bucket/bert/export/multilabel/1556817309


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['examples'] tensor_info:
        dtype: DT_STRING
        shape: (-1)
        name: foo/input_example_tensor:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['probabilities'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 6)
        name: loss/Sigmoid:0
  Method name is: tensorflow/serving/predict


In [26]:
# Execute model to understand model input format
!saved_model_cli run --dir gs://not-another-bert-bucket/bert/export/multilabel/1556817309 --tag_set serve --signature_def serving_default \
--input_examples 'examples=[{"input_ids":np.zeros((128), dtype=int).tolist(),"input_mask":np.zeros((128), dtype=int).tolist(),"label_ids":[0,0,0,0,0,0],"segment_ids":np.zeros((128), dtype=int).tolist()}]'

2019-05-02 17:38:04.702011: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-05-02 17:38:04.702290: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x55a938ebe940 executing computations on platform Host. Devices:
2019-05-02 17:38:04.702328: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
Result for output key probabilities:
[[1.9190037e-03 1.2006010e-04 4.2825087e-04 6.9388116e-05 3.9723018e-04
  1.0240604e-04]]
